In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator  
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [5]:
# initialize the initial learning rate, number of epochs to train for, and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [6]:
DIRECTORY = r"C:\Users\uniqu\Face-Mask-Detection\dataset"
CATEGORIES = ["with_mask", "without_mask"]

In [7]:
print("loading images...")

loading images...


In [8]:
data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

C:\Users\uniqu\anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [9]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

#now converting them in numpy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [10]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify = labels, random_state=42)

In [11]:
# Image Data Generator is used to generate multiple images from given input by rotating, shifting , flipping or many other things

In [12]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range = 20,
                         zoom_range = 0.15,
                         width_shift_range = 0.2,
                        height_shift_range = 0.2,
                        shear_range = 0.15,
                        horizontal_flip = True,
                        fill_mode = 'nearest')

In [12]:
# load the MobileNetV2 network, ensuring the head FC layer sets are left off
# include_top=False means not adding the fully connected layers at the top

baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,224,3)))

# 224,224 means the height and width of the input images and 3 indicates that there are 3 channel RGB(red,green,, blue) 
# which means we are inputting colored images

9412608/9406464 [==============================] - 2s 0us/step


In [13]:
# constructing the head of the model(fully connected layer) that will be placed on top of the base model

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation='softmax')(headModel)

# AveragePooling smooths out the image and hence the sharp features maynot be identified
# Flattening is converting the data into a 1-D array for inputting it to next layer
# relu can be used for non linear data or images 
# Dropout to avoid overfitting
# 2 shows two classes(with mask and without mask)

In [14]:
# place the head FC model on top of the base model (this will 
# become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [15]:
# loop over all layers in the base model and freeze them so they 
# will *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

In [18]:
# compile our model
print("compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

compiling model...


In [ ]:
# train the head of the network
print("training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS), #also flowing 'aug' image data generator as original images are less
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

training head...
Epoch 1/20
95/95 [==============================] - 607s 6s/step - loss: 0.4273 - accuracy: 0.8263 - val_loss: 0.1639 - val_accuracy: 0.9791
Epoch 2/20
95/95 [==============================] - 490s 5s/step - loss: 0.1537 - accuracy: 0.9598 - val_loss: 0.0858 - val_accuracy: 0.9896
Epoch 3/20
95/95 [==============================] - 496s 5s/step - loss: 0.1095 - accuracy: 0.9697 - val_loss: 0.0626 - val_accuracy: 0.9909
Epoch 4/20
95/95 [==============================] - 494s 5s/step - loss: 0.0806 - accuracy: 0.9806 - val_loss: 0.0519 - val_accuracy: 0.9896
Epoch 5/20
95/95 [==============================] - 520s 5s/step - loss: 0.0635 - accuracy: 0.9825 - val_loss: 0.0437 - val_accuracy: 0.9909
Epoch 6/20
95/95 [==============================] - 513s 5s/step - loss: 0.0601 - accuracy: 0.9815 - val_loss: 0.0407 - val_accuracy: 0.9909
Epoch 7/20
95/95 [==============================] - 491s 5s/step - loss: 0.0565 - accuracy: 0.9838 - val_loss: 0.0384 - val_accuracy: 0.9

In [ ]:
model.save("training_mask.h5")

In [2]:
from keras.models import load_model
model=load_model("training_mask.h5")

In [13]:
# make predictions on the testing set
print("evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

evaluating network...


In [14]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

In [15]:
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [16]:
# serialize the model to disk
print("saving mask detector model...")
model.save("mask_detector.model", save_format="h5")

saving mask detector model...


C:\Users\uniqu\anaconda3\envs\deeplearning\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
